<h1>ОБЪЕДИНЕНИЕ ТАБЛИЦ – JOIN</h1>
<p>Описание данных:<br>

installs — содержит данные об установках приложения по дням.<br>

· DeviceID — идентификатор устройства, на которое было установлено приложение;<br>
· InstallationDate — дата установки приложения;<br>
· InstallCost — цена установки приложения в рублях;<br>
· Platform — платформа, на которой было установлено приложение (iOS/ Android);<br>
· Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).<br>

events — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.<br>

· DeviceID — идентификатор устройства, на котором используется приложение;<br>
· AppPlatform — платформа, на которой используется приложение (iOS/ Android);<br>
· EventDate — дата, за которую собрана статистика;<br>
· events — количество просмотров всех товаров за этот день у этого DeviceID.<br>

checks — содержит данные о покупках пользователей в приложении по дням<br>

· UserID — идентификатор пользователя;<br>
· Rub — суммарный чек пользователя на дату;<br>
· BuyDate — дата, за которую собрана статистика.<br>

Особенность приложения заключается в том, что для просмотра товаров не нужна авторизация. До момента авторизации про пользователя известен только его DeviceID — идентификатор устройства. При этом для совершения покупки логин обязателен. На моменте авторизации пользователю присваивается UserID, и тогда мы уже знаем два его идентификатора: DeviceID (устройство) и UserID (логин). Так как на этапах установки приложения и просмотра каталога пользователь еще может быть не авторизован, там мы сохраняем только DeviceID. Но так как покупки нельзя совершить без авторизации, то покупки сохраняются только с UserID. Для того чтобы просмотры и установки можно было объединить с покупками, нам нужна таблица соответствия DeviceID к UserID, то есть таблица devices:<br>

· DeviceID — идентификатор устройства;<br>
· UserID — идентификатор пользователя.</p>

<p><b>1</b><br>
У пользователя может быть два идентификатора – UserID и DeviceID. В таблице checks есть только UserID, в остальных – только DeviceID. Во вспомогательной таблице devices есть и UserID, и DeviceID.<br>

Давайте с помощью JOIN дополним таблицу events (left) данными о UserID пользователей из таблицы devices (right). Для некоторых DeviceID не будет пары UserID из таблицы devices – подумайте, какой вид JOIN подойдет, чтобы не потерять те строки, где DeviceID есть в events, но нет в devices.<br>

Укажите UserID из первой строки результирующей таблицы, используя сортировку по убыванию по полю DeviceID.</p>

In [ ]:
select AppPlatform, UserID, ev.DeviceID
from events ev
left join devices dev
on ev.DeviceID  = dev.DeviceID
order by ev.DeviceID desc

<p><b>2</b><br>
Давайте проверим, пользователи пришедшие из какого источника совершили наибольшее число покупок. В качестве ответа выберите название Source, юзеры которого совершили больше всего покупок.<br>

Hint: Для этого используйте UserID, DeviceID и Source из соответствующих таблиц. Считать уникальные значения здесь не нужно.</p>

In [ ]:
with sr as ((select UserID, sum(Rub) from checks ch group by UserID order by sum(Rub) desc))

select Source
from installs ins
join devices dev
on ins.DeviceID = dev.DeviceID 
join sr
on sr.UserID = dev.UserID
limit 10

<p><b>3</b><br>
Теперь выясним, сколько всего уникальных юзеров что-то купили в нашем приложении.<br>

Объедините нужные таблицы, посчитайте число уникальных UserID для каждого источника (Source), и в качестве ответа укажите число пользователей, пришедших из Source_7.<br>

Hint: checks – покупки, devices – соответствие, installs – информация об источнике.</p>

In [ ]:
select Source, count(DISTINCT UserID) as Count_uniq_Users
from checks
join devices
on checks.UserID = devices.UserID
join installs
on installs.DeviceID = devices.DeviceID
where Source = 'Source_7'
group by Source

<p><b>4</b><br>
Самое время посмотреть на общую выручку, а также минимальный, максимальный и средний чек. Рассчитайте нужные показатели.</p>

In [ ]:
select Source, sum(Rub) as Revenue, min(Rub) as MinCheck, max(Rub) as MaxCheck, avg(Rub) as AvgCheck 
from checks
join devices
on checks.UserID = devices.UserID
join installs
on installs.DeviceID = devices.DeviceID
group by Source

<p><b>5</b><br>
Выведите идентификаторы устройств пользователей, которые совершили как минимум одну покупку за последний месяц (октябрь 2019). Используйте сортировку по возрастанию DeviceID и укажите минимальное значение.<br>

Hint: для извлечения месяца из даты можно использовать toMonth() или toStartOfMonth(), предварительно приведя BuyDate к типу</p>

In [ ]:
select DeviceID, count(DeviceID) as Count_Pay
from checks ch
join devices d
on d.UserID = ch.UserID
where toStartOfMonth(CAST(BuyDate as Date)) = '2019-10-01'
group by DeviceID
having count(DeviceID) > 1
limit 50

<p><b>6</b><br>
Проверим, сколько товаров (events) в среднем просматривают пользователи с разных платформ (Platform), и пришедшие из разных источников (Source). Для этого объедините таблицы events и installs, и посчитайте, сколько просмотров в среднем приходится на каждую пару платформа-канал привлечения.<br>

Отсортируйте полученную табличку по убыванию среднего числа просмотров.</p>

In [ ]:
select 
    Source,
    Platform,
    avg(events) as Average_View
from events ev
join installs ins
on ev.DeviceID = ins.DeviceID
group by Source, Platform
order by avg(events) desc

<p><b>7</b><br>
Давайте посчитаем число уникальных DeviceID в инсталлах, для которых присутствуют просмотры в таблице events с разбивкой по платформам (поле Platform). Для этого можно отобрать все строки только из таблицы installs, для которых нашлось соответствие в таблице events.</p>

In [ ]:
select count(distinct installs.DeviceID) as Uniq_Device, Platform
from installs
left join events
on events.DeviceID = installs.DeviceID
group by Platform

<p><b>8</b><br>
Давайте теперь посчитаем конверсию из инсталла в просмотр с разбивкой по платформе инсталла – в данном случае это доля DeviceID, для которых есть просмотры, от всех DeviceID в инсталлах.<br>

Для этого нужно объединить таблицы installs и events так, чтобы получить все DeviceID инсталлов и соответствующие им DeviceID из events, посчитать число уникальных DeviceID инсталлов (1) и соответствующих DeviceID из events (2) и вычислить долю (2) от (1). В качестве ответа укажите значение конверсии из инсталла в просмотр на платформе ios.</p>

In [ ]:
select count(distinct events.DeviceID) / count(distinct installs.DeviceID) as Conversion_Device
from installs
left join events
on events.DeviceID = installs.DeviceID
where Platform = 'ios'